<a href="https://colab.research.google.com/github/nrutyadoshi/NLP/blob/Classwork/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
tweets = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")

In [0]:
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz",lines=True)

In [0]:
tweets.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [0]:
tweets.Sentiment.value_counts()

neutral       2333
irrelevant    1689
negative       572
positive       519
Name: Sentiment, dtype: int64

In [0]:
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [0]:
import nltk
nltk.download("all")

In [0]:
from textblob import TextBlob

In [0]:
tweets["polarity"] = tweets.apply(lambda row: TextBlob(row.TweetText).sentiment.polarity,axis=1)
data["reviewText_polarity"] = data.apply(lambda row: TextBlob(row.reviewText).sentiment.polarity,axis=1)
data["summary_polarity"] = data.apply(lambda row: TextBlob(row.summary).sentiment.polarity,axis=1)

In [0]:
tweets["polarity"]

0       0.000000
1       0.500000
2       0.393750
3       0.541667
4       0.200000
          ...   
5108    0.000000
5109    0.500000
5110   -0.250000
5111    0.000000
5112    0.000000
Name: polarity, Length: 5113, dtype: float64

In [0]:
data["reviewText_polarity"]

0        0.180000
1        0.170939
2        0.204091
3        0.214217
4        0.350000
           ...   
53253    0.197628
53254    0.212514
53255    0.120779
53256    0.464940
53257    0.281190
Name: reviewText_polarity, Length: 53258, dtype: float64

In [0]:
data["summary_polarity"]

0       -0.025000
1        0.375000
2        0.400000
3       -0.200000
4        1.000000
           ...   
53253    0.563333
53254    0.216667
53255    0.400000
53256    0.344444
53257    0.750000
Name: summary_polarity, Length: 53258, dtype: float64

In [0]:
def label(float):
  value = ' '
  if(float==0):
    value = 'neutral'
  elif(float<0):
    value = 'negative'
  elif(float>0):
    value = 'positive'
  else:
    value = 'irrelevant'
  return (value)

In [0]:
tweets["label"] = tweets.apply(lambda row: label(row.polarity),axis=1)

In [0]:
tweets["label"].tail()

5108     neutral
5109    positive
5110    negative
5111     neutral
5112     neutral
Name: label, dtype: object

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
print(confusion_matrix(tweets.Sentiment, tweets.label))

[[   0   61 1366  262]
 [   0  234  171  167]
 [   0  201 1213  919]
 [   0   38  124  357]]


In [0]:
from nltk.sentiment import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
sid = SentimentIntensityAnalyzer()

In [0]:
tweets["analyse"] = tweets.apply(lambda row: sid.polarity_scores(row.TweetText),axis=1)
data["reviewText_analyse"] = data.apply(lambda row: sid.polarity_scores(row.reviewText),axis=1)
data["summary_analyse"] = data.apply(lambda row: sid.polarity_scores(row.summary),axis=1)

In [0]:
tweets.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,polarity,analyse,label,max
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,neu
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,"{'neg': 0.0, 'neu': 0.801, 'pos': 0.199, 'comp...",positive,neu
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,"{'neg': 0.0, 'neu': 0.591, 'pos': 0.409, 'comp...",positive,compound
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,"{'neg': 0.0, 'neu': 0.814, 'pos': 0.186, 'comp...",positive,neu
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,"{'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'comp...",positive,neu


In [0]:
def find_max(dict):
  return (max(dict,key = dict.get))

In [0]:
tweets["max"] = tweets.apply(lambda row: find_max(row.analyse),axis=1)

In [0]:
tweets.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,polarity,analyse,label,max
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,neu
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,"{'neg': 0.0, 'neu': 0.801, 'pos': 0.199, 'comp...",positive,neu
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,"{'neg': 0.0, 'neu': 0.591, 'pos': 0.409, 'comp...",positive,compound
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,"{'neg': 0.0, 'neu': 0.814, 'pos': 0.186, 'comp...",positive,neu
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,"{'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'comp...",positive,neu


In [0]:
pip install glove_python

     |████████████████████████████████| 266kB 9.7MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=702672 sha256=7910a94fe036e87c74be7a335639e3839a9cb593f22bab08e91bb3cd81dedd0f
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
from glove import Glove, Corpus

In [0]:
col = tweets.TweetText

In [101]:
column = col.apply(nltk.word_tokenize)
corpus = Corpus() 
corpus.fit(column, window=10)
glove = Glove(no_components=5, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.most_similar('amazing')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


[('Mommy_gaga', 0.9964233242919949),
 ('BGR', 0.994539429300424),
 ('iOS', 0.9923577827462627),
 ('thetechcheck', 0.9916233138357864)]